In [1]:
!pip install tensorflow==2.0.0-alpha0  --upgrade
!pip install pillow
!pip install scipy
!pip install opencv-python
!pip install matplotlib


  Using cached https://files.pythonhosted.org/packages/de/be/65c6c6ab4660ca8c0068a5fed739944f34af02b42cb727cefe0a5383dc8e/tensorflow-2.0.0a0-cp36-cp36m-macosx_10_11_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/13/82/f16063b4eed210dc2ab057930ac1da4fbe1e91b7b051a6c8370b401e6ae7/tf_estimator_nightly-1.14.0.dev2019030115-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/76/4a4c012bca5688881c18f6e04694d221b88daa5d38526a4df87d75711199/numpy-1.16.3-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/90/85/64c82949765cfb246bbdaf5aca2d55f400f792655927a017710a78445def/Keras_Applications-1.0.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/96/ba/a4702cbb6a3a485239fbe9525443446203f00771af9ac000fa3ef2788201/wheel-0.33.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/9d/6f2b4665b6

In [1]:
import math
import cv2 as cv2
import numpy as np
import asyncio
import glob


In [8]:
class Core:
    def __init__(self):
        self.kernel = 1 / 454 * np.array([[1, 4, 6, 4, 1], [4, 16, 24, 16, 4], [6, 24, 36, 24, 6], [4, 16, 24, 16, 4],
                                          [100, 4, 6, 4, 100]])
        
        self.kernel = np.array([[0.0625, 0.125, 0.0625], [0.125, 0.25, 0.125], [0.0625, 0.125, 0.0625]])
        self.nrTh = 4

    def readFile(self, fileName):
        self.img = cv2.imread(fileName)
        self.img = cv2.resize(self.img, (100, 100))
        
        # Rewrite the original image
        self.originalImg = self.img
        cv2.imwrite(fileName, self.img)

    def writeImg(self, fileName):
        cv2.imwrite(fileName, self.img)

    def parseImg(self):
        h = self.img.shape[0]
        w = self.img.shape[1]

        blank_image = np.zeros((h, w, 3), np.uint8)

        for i in range(0, self.nrTh):
            self.setNewPxThr(i,blank_image)

        for i in range(h):
            blank_image[i][0] = self.img[i][0]
            blank_image[i][1] = self.img[i][1]
            blank_image[i][-1] = self.img[i][-1]
            blank_image[i][-2] = self.img[i][-2]
            
        self.img = blank_image

    def showImg(self):
        cv2.imshow("SA", self.img)

    def multiplyPx(self, i, j, channel):
        if i <= 1 or j <= 1 or i >= self.img.shape[0] - 2 or j >= self.img.shape[1] - 2:
            return self.img[i][j][channel]

        res = 0

        for kerI in range(0, 3):
            for kerJ in range(0, 3):
                res += self.kernel[kerI, kerJ] * self.img[i - 2 + kerI, j - 2 + kerJ, channel]

        return res

    def setNewPxThr(self,thId, blank_image):
        h = self.img.shape[0]
        w = self.img.shape[1]

        linesEachProcess = math.ceil(self.img.shape[1] / self.nrTh)
        to = min(linesEachProcess * (thId + 1), int(self.img.shape[1]) - 2)

        for x in range(linesEachProcess * thId, to):
            if thId == 0:
                x+=2
            for y in range(0, h):
                for c in range(0, 3):
                    blank_image[y, x, c] = self.multiplyPx(y, x, c)


In [10]:
IMGS = glob.glob('data/train/sharp/*.jpg')
core = Core()
for i, figPath in enumerate(IMGS):
    fileName = figPath.split('/')[-1]
    core.readFile(figPath)
    core.parseImg()
    core.writeImg('data/train/blur/' + fileName)


In [11]:
print("done")    

done
